In [4]:
from collections import defaultdict, deque
from itertools import combinations

from tqdm import tqdm
import networkx as nx
import matplotlib.pyplot as plt

In [5]:
def parse_input(file):
    with open(file) as file_in:
        wires, gates = file_in.read().split('\n\n')

    wires_starting_with_z = set()

    wire_dict = {}
    for row in wires.splitlines():
        wire, value = row.split(': ')
        wire_dict[wire] = int(value)
    
    gates_dict = {}
    for row in gates.splitlines():
        rule, out = row.split(' -> ')
        in1, op, in2 = rule.split(' ')
        gates_dict[out] = (in1, op, in2)
        if out.startswith('z'):
            wires_starting_with_z.add(out)

    return wire_dict, gates_dict, wires_starting_with_z

In [6]:
def compute_output(in1, in2, op):
    if op == 'AND':
        return in1 & in2
    elif op == 'OR':
        return in1 | in2
    elif op == 'XOR':
        return in1 ^ in2

In [7]:
def run_until_all_z_done(wires, gates, wires_starting_with_z):
    while len(wires_starting_with_z & set(wires)) != len(wires_starting_with_z):
        for out, (in1, op, in2) in gates.items():
            if in1 in wires and in2 in wires:
                wires[out] = compute_output(wires[in1], wires[in2], op)
    return wires

In [8]:
def get_wires_group_as_bin(wires, group):
    wires_order = sorted([w for w in wires if w.startswith(group)], reverse=True)
    wires_values = [str(wires[w]) for w in wires_order]
    result_bin = ''.join(wires_values)
    return result_bin

In [9]:
def compute_final_number(wires):
    result_bin = get_wires_group_as_bin(wires, 'z')
    return int(result_bin, 2)

In [10]:
def main1(file):
    wires, gates, wires_starting_with_z = parse_input(file)
    wires = run_until_all_z_done(wires, gates, wires_starting_with_z)
    return compute_final_number(wires)

In [11]:
def check_addition_works(wires):
    bin_x = get_wires_group_as_bin(wires, 'x')
    bin_y = get_wires_group_as_bin(wires, 'y')
    bin_z = get_wires_group_as_bin(wires, 'z')
    
    return int(bin_x, 2) + int(bin_y, 2) == int(bin_z, 2)

In [12]:
assert main1('example1.txt') == 4
assert main1('example2.txt') == 2024

In [13]:
main1('input.txt')

36902370467952

In [13]:
def get_sequence_operations(out, gates):
    if out not in gates:
        # Base out: the output is an input node (xnn or ynn)
        return out

    input1, operation, input2 = gates[out]
    # Recursively substitute inputs
    expr1 = get_sequence_operations(input1, gates)
    expr2 = get_sequence_operations(input2, gates)
    # Construct the expression
    return f"({expr1} {operation} {expr2})"

In [48]:
wires, gates, wires_starting_with_z = parse_input('input.txt')

for out in sorted(wires_starting_with_z):
    seq2out = get_sequence_operations(out, gates)
    print(seq2out)

(x00 XOR y00)
((x00 AND y00) XOR (y01 XOR x01))
((x02 XOR y02) XOR (((x00 AND y00) AND (y01 XOR x01)) OR (y01 AND x01)))
((((((x00 AND y00) AND (y01 XOR x01)) OR (y01 AND x01)) AND (x02 XOR y02)) OR (y02 AND x02)) XOR (y03 XOR x03))
((y04 XOR x04) XOR ((x03 AND y03) OR ((y03 XOR x03) AND (((((x00 AND y00) AND (y01 XOR x01)) OR (y01 AND x01)) AND (x02 XOR y02)) OR (y02 AND x02)))))
(((x04 AND y04) OR ((y04 XOR x04) AND ((x03 AND y03) OR ((y03 XOR x03) AND (((((x00 AND y00) AND (y01 XOR x01)) OR (y01 AND x01)) AND (x02 XOR y02)) OR (y02 AND x02)))))) XOR (y05 XOR x05))
((((y05 XOR x05) AND ((x04 AND y04) OR ((y04 XOR x04) AND ((x03 AND y03) OR ((y03 XOR x03) AND (((((x00 AND y00) AND (y01 XOR x01)) OR (y01 AND x01)) AND (x02 XOR y02)) OR (y02 AND x02))))))) OR (x05 AND y05)) XOR (y06 XOR x06))
((((y06 XOR x06) AND (((y05 XOR x05) AND ((x04 AND y04) OR ((y04 XOR x04) AND ((x03 AND y03) OR ((y03 XOR x03) AND (((((x00 AND y00) AND (y01 XOR x01)) OR (y01 AND x01)) AND (x02 XOR y02)) OR (y02 

In [30]:
wires, gates, wires_starting_with_z = parse_input('input.txt')
wires = run_until_all_z_done(wires, gates, wires_starting_with_z)

x, y = wires['x00'], wires['y00']
for i in range(45):
    x, y = wires[f'x{i:02d}'], wires[f'y{i:02d}']
    partial_sum = x ^ y
    if i == 0:
        assert wires[f'z{i:02d}'] == partial_sum
        carry = x & y
    else:
        assert wires[f'z{i:02d}'] == partial_sum ^ carry, i
        carry = (carry & partial_sum) | (x & y)

AssertionError: 10

0

In [40]:
z

0